In [1]:
!pip install pymorphy2 
!pip install --upgrade datasets
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/55.5 kB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00


   ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/8.2 MB 63.4 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 8.1/8.2 MB 117.5 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.2 MB/s eta 0:00:00


...

  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.1


    Uninstalling datasets-3.0.1:


      Successfully uninstalled datasets-3.0.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 317.4/320.7 kB 10.8 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.1 MB/s eta 0:00:00


In [2]:
import pandas as pd

import torch
from transformers import AutoModelForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from peft import LoraConfig

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import datasets

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words("russian")

import re

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import optuna
import os

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def clean_text(text):
    # Remove URLs, numbers, and special characters
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # URLs
    text = re.sub(r'\d+', '', text)  # Numbers
    text = re.sub(r'[^\w\s]', '', text)  # Special characters
    return text

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    # remove stop words
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # lemmatize
    def lemmatize(tokens):
        return [morph.parse(word)[0].normal_form for word in tokens]
    tokens = lemmatize(tokens)
    return ' '.join(tokens)

def all_preprocessing(df):
    df['text'] = df['text'].apply(clean_text)
    df['text'] = df['text'].apply(preprocess_text)
    return df

In [4]:
import wandb

wandb.login(key='b123af3ff1bc7e54569d0976c6405a5b3b6d2902')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
df = pd.read_csv('/kaggle/input/wb_winter_24/train.csv', index_col='ID')
#df = all_preprocessing(df)
df.head()

,text,label
ID,,
0,"Брюки отличные, качественные, но к сожалению к...",0
1,"Отличный аппарат, в комплекте кабель и работет...",0
2,Супер 👍 спасибо большое,0
3,Получил быстро данные наушники! К наушникам пр...,0
4,Всё дошло в целости и сохранности),0


# Prepare dataset

In [6]:
# Split dataset into training and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), 
    df['label'].tolist(), 
    test_size=0.05,
    shuffle=True,
    stratify=df['label'].tolist(),
    random_state=42
)
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')

# Create a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Prepare datasets
train_dataset, val_dataset = CustomDataset(train_texts, train_labels, tokenizer), CustomDataset(val_texts, val_labels, tokenizer)

# Define compute metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='weighted'),
        'precision': precision_score(p.label_ids, preds, average='weighted'),
        'recall': recall_score(p.label_ids, preds, average='weighted')
    }

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

# BERT Finetuning with Optuna

In [7]:
def model_init(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    num_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    batch_size = trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128])
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.05, 0.3)
    weight_decay = trial.suggest_float("weight_decay", 0.00, 0.3)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.5, 1.0)
    lr_scheduler_type = trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine", "constant_with_warmup"])

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = AutoModelForSequenceClassification.from_pretrained(
        'blanchefort/rubert-base-cased-sentiment', num_labels=2, ignore_mismatched_sizes=True  # Assuming binary classification
    )
    model.to(device)
    
    model_dir = f"/kaggle/working/trial_{trial.number}"
    training_args = TrainingArguments(
        output_dir=model_dir,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        num_train_epochs=num_epochs,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
        max_grad_norm=max_grad_norm,
        lr_scheduler_type=lr_scheduler_type,
        fp16=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        save_total_limit=1,
        metric_for_best_model="f1"
    )

    return model, training_args

In [8]:
def remove_unnecessary_checkpoints(model_dir):
    # Path to the best checkpoint (this will be automatically saved by Trainer)
    best_checkpoint_dir = f"{model_dir}/checkpoint-best"

    # List all directories in the output_dir
    checkpoints = [f for f in os.listdir(output_dir) if f.startswith("checkpoint")]

    # Remove all checkpoints except the best one
    for checkpoint in checkpoints:
        checkpoint_path = os.path.join(output_dir, checkpoint)

        # Skip the best checkpoint
        if checkpoint_path != best_checkpoint_dir:
            print(f"Deleting checkpoint: {checkpoint_path}")
            shutil.rmtree(checkpoint_path)

def objective(trial):
    model, training_args = model_init(trial)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )
    trainer.train()

    # Remove not best checkpoints
    model_dir = f"/kaggle/working/trial_{trial.number}"
    #remove_unnecessary_checkpoints(model_dir)
    
    eval_results = trainer.evaluate()
    return eval_results["eval_f1"]

# Set up Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

best_params, best_trial = study.best_params, study.best_trial
# Print best hyperparameters
print("Best hyperparameters found:", best_params)
print("Best trial found:", best_trial)

[I 2024-11-13 20:40:01,277] A new study created in memory with name: no-name-1a34c4c3-97cb-4610-857a-c2e575debbcb


model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at blanchefort/rubert-base-cased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.111700,0.108249,0.967855,0.967938,0.968030,0.967855
2,0.056400,0.062171,0.984677,0.984527,0.984513,0.984677
3,0.044700,0.050700,0.987009,0.986932,0.986906,0.987009
4,0.027400,0.054204,0.986676,0.986664,0.986653,0.986676
5,0.022800,0.061949,0.985926,0.985968,0.986024,0.985926


[I 2024-11-14 03:56:07,398] Trial 0 finished with value: 0.986932305051494 and parameters: {'learning_rate': 1.506384070586987e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 64, 'warmup_ratio': 0.2831308254995163, 'weight_decay': 0.04336209789206402, 'max_grad_norm': 0.9719496496056579, 'lr_scheduler_type': 'cosine'}. Best is trial 0 with value: 0.986932305051494.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at blanchefort/rubert-base-cased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.098800,0.102534,0.974184,0.973970,0.973850,0.974184
2,0.065600,0.065447,0.984594,0.984591,0.984589,0.984594
3,0.052800,0.058467,0.986176,0.986184,0.986192,0.986176


In [ ]:
test_df = pd.read_csv('/kaggle/input/wb_winter_24/test.csv', index_col='ID')
#test_df = all_preprocessing(test_df)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(
    f"/kaggle/working/trial_{best_trial.number}", num_labels=2, ignore_mismatched_sizes=True  # Assuming binary classification
)
model.to(device)
model.eval()  # Set model to evaluation mode

def predict(text):
    # Tokenize input text
    inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    return predictions.item()  # Return predicted label (0 or 1 for binary classification)

test_df['label'] = test_df['text'].apply(predict)
test_df = test_df.drop(columns='text')
test_df.to_csv('/kaggle/working/test_predicted.csv')